In [ ]:
# https://towardsdatascience.com/a-quick-introduction-on-granger-causality-testing-for-time-series-analysis-7113dc9420d2
# A Quick Introduction On Granger Causality Testing For Time Series Analysis 

#Could we use today’s Apple’s stock price to predict tomorrow’s Tesla’s stock price? \
# If this is true, our statement will be Apple’s stock price Granger causes Tesla’s stock price. 
#If this is not true, we say Apple’s stock price does not Granger cause Tesla’s stock price.


In [2]:
folder = r'D:\18-DS\data\Yahoostockdata'
import pandas as pd
df_apple = pd.read_csv(folder+'/AAPL.csv')
df_walmart = pd.read_csv(folder+'/WMT.csv')
df_tesla = pd.read_csv(folder+'/TSLA.csv')

df = pd.merge(df_apple[['Date', 'Adj Close']], df_walmart[['Date', 'Adj Close']], on='Date', how='right').rename(columns = {'Adj Close_x':'apple', 'Adj Close_y':'walmart'})
df = df.merge(df_tesla[['Date', 'Adj Close']], on='Date', how='right').rename(columns={'Adj Close':'tesla'})

df['Date'] =  pd.to_datetime(df['Date'])
df = df.set_index('Date').rename_axis('company', axis=1)

df.head()

company,apple,walmart,tesla
Date,,,
2010-06-30,7.713137,36.875237,4.766
2010-07-01,7.619610,37.082359,4.392
2010-07-02,7.572384,36.821545,3.840
2010-07-06,7.624209,37.258797,3.222
2010-07-07,7.932084,37.527287,3.160


In [3]:
import plotly.express as px

fig = px.line(df, facet_col="company", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

In [4]:
fig = px.area(df, facet_col='company', facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

## ADF Test for Stationarity

In [ ]:

# It can be used to help us understand whether the time series is stationary or not.
# It assumes timeseries is not stationary.
# Null hypothesis: If H0 is failed to be rejected, it suggests the time series is not stationarity.
# Alternative hypothesis: The null hypothesis is rejected, it suggests the time series is stationary.
    

In [5]:
n_obs = 20
df_train, df_test = df[0:-n_obs], df[-n_obs:]

from statsmodels.tsa.stattools import adfuller

def adf_test(df):
    result = adfuller(df.values)
    print('ADF Statistics: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
        
print('ADF Test: Apple time series')
adf_test(df_train['apple'])
print('ADF Test: Walmart time series')
adf_test(df_train['walmart'])
print('ADF Test: Tesla time series')
adf_test(df_train['tesla'])

ADF Test: Apple time series
ADF Statistics: 3.118747
p-value: 1.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Walmart time series
ADF Statistics: 1.869043
p-value: 0.998477
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Tesla time series
ADF Statistics: 7.460463
p-value: 1.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


In [ ]:
# The p-values are all well above the 0.05 alpha level, we cannot reject the null hypothesis. 
# So the three time series are not stationary.

# KPSS Test for Stationary

In [ ]:

#if a time series is stationary around a mean or linear trend, or is non-stationary due to a unit root.
#Null hypothesis: The time series is stationary
#Alternative hypothesis: The time series is not stationary
    

In [6]:
from statsmodels.tsa.stattools import kpss

def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
        
print('KPSS Test: Apple time series')
kpss_test(df_train['apple'])
print('KPSS Test: Walmart time series')
kpss_test(df_train['walmart'])
print('KPSS Test: Tesla time series')
kpss_test(df_train['tesla'])

KPSS Test: Apple time series
KPSS Statistic: 6.229542577734188
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Walmart time series
KPSS Statistic: 6.755196802273019
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Tesla time series
KPSS Statistic: 3.469255176231092
p-value: 0.01
num lags: 30
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




In [ ]:
# The p-value are all less than 0.05 alpha level, therefore, we can reject the null hypothesis and 
# derive that the three time series are not stationary.

# Difference Method

In [7]:

df_train_transformed = df_train.diff().dropna()

fig = px.line(df_train_transformed, facet_col="company", facet_col_wrap=1)
fig.update_yaxes(matches=None)
fig.show()

# ADF Test Again

In [8]:
print('ADF Test: Apple time series transformed')
adf_test(df_train_transformed['apple'])
print('ADF Test: Walmart time series transformed')
adf_test(df_train_transformed['walmart'])
print('ADF Test: Tesla time series transformed')
adf_test(df_train_transformed['tesla'])

ADF Test: Apple time series transformed
ADF Statistics: -8.869680
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Walmart time series transformed
ADF Statistics: -10.726903
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567
ADF Test: Tesla time series transformed
ADF Statistics: -7.235143
p-value: 0.000000
Critical values:
	1%: -3.433
	5%: -2.863
	10%: -2.567


In [9]:
print('KPSS Test: Apple time series transformed')
kpss_test(df_train_transformed['apple'])
print('KPSS Test: Walmart time series transformed')
kpss_test(df_train_transformed['walmart'])
print('KPSS Test: Tesla time series transformed')
kpss_test(df_train_transformed['tesla'])

KPSS Test: Apple time series transformed
KPSS Statistic: 0.7906606342295729
p-value: 0.01
num lags: 6
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Walmart time series transformed
KPSS Statistic: 0.3742199707329828
p-value: 0.08826725399440397
num lags: 15
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
KPSS Test: Tesla time series transformed
KPSS Statistic: 1.1596814164611662
p-value: 0.01
num lags: 6
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739


d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.


d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:2018: InterpolationWarning:

The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.




In [ ]:
# VAR Model
# The VAR class assumes that the passed time series are stationary. Non-stationary or trending data can 
# often be transformed to be stationary by first-differencing or some other method.

In [10]:
from statsmodels.tsa.api import VAR

var_model = VAR(df_train_transformed)
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    result = var_model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

d:\Users\admin\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



Lag Order = 1
AIC :  2.5114692680446997
BIC :  2.538333502798189
FPE :  12.32302267657725
HQIC:  2.521198477873641 

Lag Order = 2
AIC :  2.5027701795332065
BIC :  2.5497974663184073
FPE :  12.21628890782523
HQIC:  2.519802014365096 

Lag Order = 3
AIC :  2.485715099818324
BIC :  2.5529182040150817
FPE :  12.00970665699808
HQIC:  2.510054466235962 

Lag Order = 4
AIC :  2.482243371994503
BIC :  2.5696350720988734
FPE :  11.968086111079794
HQIC:  2.5138951817691706 

Lag Order = 5
AIC :  2.462627687319315
BIC :  2.5702207749620682
FPE :  11.735613921388923
HQIC:  2.5015968574182295 

Lag Order = 6
AIC :  2.456558733548684
BIC :  2.584366013513841
FPE :  11.664610309271902
HQIC:  2.5028501861424677 

Lag Order = 7
AIC :  2.4448360162749614
BIC :  2.592870306518366
FPE :  11.528672630244564
HQIC:  2.4984546787451274 

Lag Order = 8
AIC :  2.394451102819805
BIC :  2.5627252344877265
FPE :  10.962198434381632
HQIC:  2.4554019077662494 

Lag Order = 9
AIC :  2.367483255070653
BIC :  2.556010

In [11]:
results = var_model.fit(maxlags=15, ic='aic')
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 07, Aug, 2022
Time:                     09:39:14
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    2.56183
Nobs:                     2609.00    HQIC:                   2.36393
Log likelihood:          -13905.1    FPE:                    9.50223
AIC:                      2.25152    Det(Omega_mle):         9.01684
--------------------------------------------------------------------
Results for equation apple
                 coefficient       std. error           t-stat            prob
------------------------------------------------------------------------------
const               0.052049         0.016780            3.102           0.002
L1.apple           -0.110754         0.022774           -4.863           0.000
L1.walmart         -0.077754         0.017598           -4.418      

# Durbin-Watson Statistic

In [23]:
# a measure of autocorrelation in residuals from regression analysis.

In [12]:
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(results.resid)

for col, val in zip(df.columns, out):
    print(col, ':', round(val, 2))

apple : 1.99
walmart : 2.0
tesla : 1.99


In [ ]:
# A value of 2.0 means that there is no autocorrelation detected in the residuals.

In [ ]:
# Granger Causality Test

In [26]:
from statsmodels.tsa.stattools import grangercausalitytests
import numpy as np

maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df_train_transformed, variables = df_train_transformed.columns)

,apple_x,walmart_x,tesla_x
apple_y,1.0,0.0,0.0
walmart_y,0.0,1.0,0.0
tesla_y,0.0,0.0,1.0


In [ ]:
#The row are the response (y) and the columns are the predictors (x). 
#If a given p-value is < significance level (0.05), for example, take the value 0.0 in (row 1, column 2), 
#we can reject the null hypothesis and conclude that walmart_x Granger causes apple_y. 
#Likewise, the 0.0 in (row 2, column 1) refers to walmart_y Granger causes apple_x.

# Forecasting

In [13]:
lag_order = results.k_ar

df_input = df_train_transformed.values[-lag_order:]
df_forecast = results.forecast(y=df_input, steps=n_obs)
df_forecast = (pd.DataFrame(df_forecast, index=df_test.index, columns=df_test.columns + '_pred'))

def invert_transformation(df, pred):
    forecast = df_forecast.copy()
    columns = df.columns
    for col in columns:
        forecast[str(col)+'_pred'] = df[col].iloc[-1] + forecast[str(col)+'_pred'].cumsum()
    return forecast
output = invert_transformation(df_train, df_forecast)

combined = pd.concat([output['apple_pred'], df_test['apple'], output['walmart_pred'], df_test['walmart'], output['tesla_pred'], df_test['tesla']], axis=1)


In [14]:
combined= combined.dropna()

In [15]:
combined

,apple_pred,apple,walmart_pred,walmart,tesla_pred,tesla
Date,,,,,,
2020-12-02,121.665458,122.504417,150.972459,148.218597,582.107060,568.820007
2020-12-03,123.369760,122.365074,151.135015,147.017258,584.281736,593.380005
2020-12-04,124.858525,121.678307,152.329149,146.633209,582.376330,599.039978
2020-12-07,123.687687,123.171288,152.917820,145.845459,576.824104,641.760010
2020-12-08,121.523853,123.798340,152.540877,147.164963,566.632604,649.880005
2020-12-09,121.743443,121.210495,153.641162,146.002991,564.534095,604.479980
2020-12-10,121.079693,122.663666,153.596580,145.321060,561.828539,627.070007
2020-12-11,120.221412,121.837555,153.337200,145.281540,556.434406,609.989990
2020-12-14,117.254231,121.210495,152.503449,143.947311,550.889569,639.830017


In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


rmse = mean_squared_error(combined['apple'], combined['apple_pred'], squared=False)
mae = mean_absolute_error(combined['apple'], combined['apple_pred'])


print('Forecast accuracy of Apple')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))


Forecast accuracy of Apple
RMSE:  6.49
MAE:  4.41


In [45]:
rmse = mean_squared_error(combined['walmart_pred'], combined['walmart'], squared=False)
mae = mean_absolute_error(combined['walmart_pred'], combined['walmart'])

print('Forecast accuracy of Walmart')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Walmart
RMSE:  6.74
MAE:  6.53


In [44]:
rmse = mean_squared_error(combined['tesla_pred'], combined['tesla'], squared=False)
mae = mean_absolute_error(combined['tesla_pred'], combined['tesla'])

print('Forecast accuracy of Tesla')
print('RMSE: ', round(rmse,2))
print('MAE: ', round(mae,2))

Forecast accuracy of Tesla
RMSE:  69.18
MAE:  60.62
